In [ ]:
import sys
from os import listdir, makedirs
from os.path import join, exists

import numpy as np
import torch
import spacy
import codecs
import xmltodict
import pickle
from tqdm import tqdm
from allennlp.modules.elmo import Elmo, batch_to_ids

In [ ]:
WORK_DIR = "../Data/bioc_FH_training/"

torch.set_num_threads(8)

In [ ]:
def read_xml(path, f):
    """
    Get Observations and FamaliMembers from xml file.
    """
    
    with open(join(path, f),'r') as f:
        content = f.read()
        content = xmltodict.parse(content)

    Obs, Fam = [], []
    for entity in content['data']['annotations']['entity']: # get `entity`s from xml
        spans = []
        for span in entity['span'].split(';'):
            spans.append(span.split(','))

        if entity['type'] == 'Observation':
            Obs.extend(spans)
        elif entity['type'] == 'FamilyMember':
            Fam.extend(spans)

    return {'OBS': Obs, 'FAM': Fam}

In [ ]:
nlp = spacy.load('en_core_web_lg')

options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 2, dropout=0)

In [ ]:
def get_token_ent_type(token, entities):
    """
    Get the entity type of a token.
    """

    for type_, entities_ in entities.items():
        for entity in entities_:
            if token.idx >= int(entity[0]) and token.idx + len(token) <= int(entity[1]):
                # if a token is in the span of a annotation with a type, then the token will be of that type
                print(token, type_)
                return type_

In [ ]:
def get_doc_label(doc, entities):
    """
    Get labels of all tokens in a document.
    Each token is labeled as [entity_type, POS_type, dependency]
    """
    
    doc_label = []
    
    for sent in doc.sents:
        sent_label = []

        for token in sent:
            ent_type = get_token_ent_type(token, entities)
            if ent_type:
                token.ent_type_ = ent_type
            else:
                print(token, 'O')

            if not token.ent_type_ in entd.keys():
                entd[token.ent_type_] = len(entd)

            if not token.pos_ in posd.keys():
                posd[token.pos_] = len(posd)

            if not token.dep_ in depd.keys():
                depd[token.dep_] = len(depd)

            sent_label.append([entd[token.ent_type_],posd[token.pos_],depd[token.dep_]])

        doc_label.append(sent_label)
        
    return doc_label

In [ ]:
txtfiles = [f for f in listdir(WORK_DIR) if f.endswith('.txt')]
xmlfiles = [f[:-3]+'xml' for f in txtfiles]

corpus = []
labels = []

entd = dict()
posd = dict()
depd = dict()
entd['<PAD>'] = len(entd)
posd['<PAD>'] = len(posd)
depd['<PAD>'] = len(depd)

for findex, txtfile in enumerate(txtfiles):
    
    entities = read_xml(WORK_DIR, xmlfiles[findex])
    #print(entities) ###
    
    with open(join(WORK_DIR, txtfile),'r') as f:
        content = f.read()
        doc = nlp(content)

    labels.append(get_doc_label(doc, entities))

    doc_sent = list(map(lambda sent: list(map(lambda x: x.text, sent)), doc.sents)) # sents to a list of lists of tokens
    corpus.append(doc_sent)
    doc_emb = elmo(batch_to_ids(doc_sent)) # generate embemdding !! - list of lists [[embd_of_word, ], ]
    assert doc_emb['mask'].sum().item() == len(doc)
    with open(join(WORK_DIR, txtfile[:-3] + 'pkl'), 'wb') as f:
        pickle.dump(doc_emb, f)

In [ ]:
print('labels:', labels[0][0])
print('\ncorpus:', corpus[0][0])

In [ ]:
np.save(join(WORK_DIR, 'label'), np.array(labels))
with open(join(WORK_DIR, 'dict.pkl'),'wb') as f:
    pickle.dump([entd, posd, depd], f)

In [ ]:
doc_emb['elmo_representations'][0][0][0].size()

In [ ]:
print(entd)
# print(posd)
# print(depd)